# Accessing ArangoDB

In [ ]:
import os
from dotenv import load_dotenv
from src.database_manager_factory import DatabaseManagerFactory

In [ ]:
load_dotenv()

config = {'db_name': os.getenv("DB_NAME"),
          'db_username': os.getenv("DB_USERNAME"),
          'db_password': os.getenv("DB_PASSWORD"),
          'db_system': os.getenv("DB_SYSTEM"),
          'db_config_file_path': os.getenv("DB_CONFIG_PATH"),
          'db_queries_file_path': os.getenv("DB_QUERIES_PATH")}

config['db_system']

In [ ]:
# create database client
#
db = DatabaseManagerFactory.get_database_manager(username=config['db_username'],
                                                 database_system=config['db_system'],
                                                 database_name=config['db_name'],
                                                 config_file_path=config['db_config_file_path'],
                                                 queries_file_path=config['db_queries_file_path'])

### Getting collections

In [ ]:
collections = db.get_collections()
collections

### Checking collection exists

In [ ]:
db.has_collection('Client')

### Creating collections

In [ ]:
db.create_collection(collection_name='Client', parameters = {'is_edge_collection': False})

db.create_collection(collection_name='Sector', parameters = {'is_edge_collection': False})

### Inserting docs & nodes

In [ ]:
docs_to_add = [{'_key': 'ABC_LTD', '_node_type': 'Client', '_node_uid': 'ABC_LTD'},
               {'_key': 'DEF_LTD', '_node_type': 'Client', '_node_uid': 'DEF_LTD'},
              ]

db.insert_many_documents(collection_name='Client', documents=docs_to_add)

docs_to_add = [{'_key': 'Construction', '_node_type': 'Sector', '_node_uid': 'Construction'}]
db.insert_many_documents(collection_name='Sector', documents=docs_to_add)

### Running pre-canned AQL

In [ ]:
cursor = db.execute_query('get_docs_in_collection', parameters={'collection': {'value': 'Client','type': 'collection'}})

for record in cursor:
    print(record)

### Inserting edges

In [ ]:
db.create_collection(collection_name='has_sector', parameters = {'is_edge_collection': True})

docs_to_add = [{'_key': 'Client:ABC_LTD::has_sector::Sector:Construction', 
                '_from': 'Client/ABC_LTD',
                '_to':'Sector/Construction',
                '_edge_type': 'has_sector', 
                '_edge_uid': None},
               
               {'_key': 'Client:DEF_LTD::has_sector::Sector:Construction', 
                '_from': 'Client/DEF_LTD',
                '_to':'Sector/Construction',
                '_edge_type': 'has_sector', 
                '_edge_uid': None},
              ]

db.insert_many_documents(collection_name='has_sector', documents=docs_to_add)


### WARNING! Delete all collections

In [ ]:
for col in db.get_collections():
    db.delete_collection(col['name'])
    